In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import  SVC, NuSVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
import yandex_evaluation as evaluation
import xgboost as xgb
%pylab inline

In [ ]:
tr = pd.read_csv('input/training.csv', index_col='id')
variables = tr.columns[:-4]

In [71]:
m = xgb.XGBClassifier(
    learning_rate=0.0065,
    max_depth=7,
    n_estimators=500,
    nthread=4,
    min_child_weight=10,
    gamma=0.01,
    colsample_bytree=0.1)
m.fit(tr[variables], tr.signal)
check_agreement = pd.read_csv('input/check_agreement.csv', index_col='id')
agreement_probs = m.predict_proba(check_agreement[variables])[:, 1]
ks = evaluation.compute_ks(
    agreement_probs[check_agreement['signal'].values == 0],
    agreement_probs[check_agreement['signal'].values == 1],
    check_agreement[check_agreement['signal'] == 0]['weight'].values,
    check_agreement[check_agreement['signal'] == 1]['weight'].values)
print 'KS metric', ks, ks < 0.09

KS metric 0.0889257931193 True


In [72]:
check_correlation = pd.read_csv('input/check_correlation.csv', index_col='id')
correlation_probs = m.predict_proba(check_correlation[variables])[:, 1]
cvm = evaluation.compute_cvm(correlation_probs, check_correlation['mass'])
print 'CvM metric', cvm, cvm < 0.002
tr_eval = tr[tr['min_ANNmuon'] > 0.4]
tr_probs = m.predict_proba(tr_eval[variables])[:, 1]
AUC = evaluation.roc_auc_truncated(tr_eval['signal'], tr_probs)
print 'AUC', AUC
test = pd.read_csv('input/test.csv', index_col='id')
result = pd.DataFrame({'id': test.index})
result['prediction'] = m.predict_proba(test[variables])[:, 1]
result.to_csv('output/submission_004.csv', index=False, sep=',')

CvM metric 0.000902710593782 True
AUC 0.986000124211
